In [2]:
from functools import partial
from sqlite3 import connect

import astroplan as ap
import astropy.units as u
import numpy as np

from astropaul.database import database_connection
import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt
import astropaul.html as html
import astropaul.phase as ph
import astropaul.priority as pr

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
session = tlc.ObservingSession(ap.Observer.at_site("LBT"))
session.add_day_range("2025-09-29", "2025-10-02")
# session.add_day_range("2025-10-18", "2025-10-22")
# session.add_day_range("2025-11-14", "2025-11-20")
# session.add_day_range("2025-12-19", "2025-12-23")
# session.add_day_range("2026-01-14", "2026-01-18")
# session.add_day_range("2026-01-27", "2026-01-31")

science_targets = {"QuadEB", "SextEB"}

phase_event_defs = [
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.0)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.05)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.18)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.32)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.45)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.55)),
    ph.PhaseEventDef("R", partial(ph.calc_time_of_phase, phase=0.68)),
    ph.PhaseEventDef("B", partial(ph.calc_time_of_phase, phase=0.82)),
    ph.PhaseEventDef("T", partial(ph.calc_time_of_phase, phase=0.95)),
]

pepsi_args = {
    "fiber": "300",
    "cd_blue": 3,
    "cd_red": 6,
    "snr": 100,
    "binocular": True,
    "priority": "(see grid)",
}

name = f"LBT Observing {session.time_range[0].iso[:10]}"

html_dir = html.output_directory() / name
html.clear_directory(html_dir)

with database_connection() as conn:
    creator = tlc.TargetListCreator(name=name, connection=conn, phase_event_defs=phase_event_defs)
    creator.steps = [
        tlc.add_targets,
        # partial(tlc.filter_targets, criteria=lambda df: (df["Target Type"].isin(["QuadEB", "SextEB"]))),
        tlc.ancillary_data_from_tess,
        partial(tlc.filter_targets, inverse=True, criteria=lambda df: df["Teff"].isna()),
        tlc.add_lists,
        partial(
            tlc.filter_targets,
            inverse=True,
            criteria=lambda df: (df["Target Type"].isin(science_targets)) & (~df["List LBT 2025B"]),
        ),
        partial(tlc.add_database_table, table_name="pepsi_observations"),
        partial(tlc.filter_targets, criteria=lambda df: (df["Num PEPSI Observations"] > 0)),
        tlc.add_tess_catalog_associations,
        partial(tlc.add_database_table, table_name="ephemerides"),
        # speckle
        partial(tlc.add_database_table, table_name="dssi_observations"),
        partial(
            tlc.add_system_configuration,
            table_name="DSSI Observations",
            time_column="Mid JD",
            eclipse_table="SIDE Observations",
        ),
        partial(tlc.add_database_table, table_name="speckle_detections"),
        partial(tlc.add_database_table, table_name="zorroalopeke_observations"),
        # pepsi
        partial(tlc.add_columns_from_sql, table_name="rv_calibration_targets", primary_cols=["RV"]),
        tlc.add_pepsi_evaluations,
        tlc.add_rv_status,
        partial(
            tlc.add_system_configuration,
            table_name="PEPSI Observations",
            time_column="Mid JD",
            # synthetic_phase_percent=0.02,
            # eclipse_table="SIDE Observations",
        ),
        partial(tlc.add_observability, observing_session=session, calc_moon_distance=True),
        partial(tlc.filter_targets, criteria=lambda df: (df["Observable Any Night"])),
        partial(lbt.assign_rv_standards, target_types=science_targets),
        partial(lbt.add_pepsi_params, **pepsi_args),
        partial(tlc.filter_targets, criteria=lambda df: df["PEPSI exp_time"] < 600),
    ]
    tl = creator.calculate(verbose=False)
tl.target_list["PEPSI notes"] = [type.replace("QuadEB", "Science Target") for type in tl.target_list["Target Type"]]
readme = lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session)

print(tl.summarize())
print(f"{np.sum(tl.target_list["PEPSI exp_time"])/60:.1f} minutes")
tl.target_list

Name: LBT Observing 2025-09-29
Criteria
    Inverse of: lambda df: df["Teff"].isna()
    Inverse of: criteria=lambda df: (df["Target Type"].isin({'SextEB', 'QuadEB'})) & (~df["List LBT 2025B"]),
    lambda df: (df["Num PEPSI Observations"] > 0)
    AltitudeConstraint: {'min': 30.0, 'max': 80.0, 'boolean_constraint': True}
    lambda df: (df["Observable Any Night"])
    lambda df: df["PEPSI exp_time"] < 600
18 targets:
      16 QuadEB
       1 RV Standard
       1 Telluric Standard
Column Count (primary, secondary):
    Target: (3, 4)
    TESS Data: (2, 123)
    List: (0, 14)
    Count: (6, 0)
    RV Calibration Targets: (1, 2)
    Observable: (5, 16)
    PEPSI : (3, 6)
Associated tables:
     622 rows,  2 columns: List Memberships
     118 rows, 19 columns: PEPSI Observations
      77 rows,  3 columns: Catalog Membership
      55 rows,  7 columns: Ephemerides
      56 rows, 12 columns: DSSI Observations
      13 rows,  6 columns: SIDE Observations
      56 rows, 11 columns: Speckle Det

,Target Name,RA,Dec,RA HMS,Dec DMS,Target Type,Source,Vmag,Teff,ID,...,RV Standard,PEPSI fiber,PEPSI cd_blue,PEPSI cd_blue_num_exp,PEPSI cd_red,PEPSI cd_red_num_exp,PEPSI snr,PEPSI exp_time,PEPSI priority,PEPSI notes
0,TIC 123098844,279.572833,44.698600,18:38:17.48,+44:41:54.96,QuadEB,Kostov 2022 arXiv:2202.05790,11.13600,6761.00,123098844.0,...,HD 65583,300,3,1,6,1,100,189,(see grid),Science Target
1,TIC 260056937,63.922208,47.422197,04:15:41.33,+47:25:19.91,QuadEB,Kostov 2022 arXiv:2202.05790,10.30200,8030.00,260056937.0,...,HD 65583,300,3,1,6,1,100,81,(see grid),Science Target
2,TIC 278352276,307.503625,48.607056,20:30:00.87,+48:36:25.4,QuadEB,Kostov 2022 arXiv:2202.05790,10.38700,7156.00,278352276.0,...,HD 65583,300,3,1,6,1,100,94,(see grid),Science Target
3,TIC 286470992,45.330708,60.572294,03:01:19.37,+60:34:20.26,QuadEB,Kostov 2022 arXiv:2202.05790,10.32500,8693.00,286470992.0,...,HD 65583,300,3,1,6,1,100,81,(see grid),Science Target
4,TIC 307119043,14.827542,51.221642,00:59:18.61,+51:13:17.91,QuadEB,Kostov 2022 arXiv:2202.05790,9.94000,7815.00,307119043.0,...,HD 65583,300,3,1,6,1,100,61,(see grid),Science Target
5,TIC 317863971,110.567500,3.031925,07:22:16.2,+03:01:54.93,QuadEB,Kostov 2022 arXiv:2202.05790,10.30600,8506.00,317863971.0,...,HD 65583,300,3,1,6,1,100,81,(see grid),Science Target
6,TIC 322727163,309.716625,50.466819,20:38:51.99,+50:28:00.55,QuadEB,Kostov 2022 arXiv:2202.05790,10.99700,7876.56,322727163.0,...,HD 65583,300,3,1,6,1,100,154,(see grid),Science Target
7,TIC 367448265,78.382458,35.653053,05:13:31.79,+35:39:10.99,QuadEB,Kostov 2022 arXiv:2202.05790,7.90000,9212.00,367448265.0,...,HD 65583,300,3,1,6,1,100,60,(see grid),Science Target
8,TIC 389836747,23.295458,61.585308,01:33:10.91,+61:35:07.11,QuadEB,Kostov 2022 arXiv:2202.05790,10.71800,8874.00,389836747.0,...,HD 65583,300,3,1,6,1,100,116,(see grid),Science Target
9,TIC 392229331,54.767917,61.064203,03:39:04.3,+61:03:51.13,QuadEB,Kostov 2022 arXiv:2202.05790,10.61600,8028.00,392229331.0,...,HD 65583,300,3,1,6,1,100,108,(see grid),Science Target


In [16]:
illumination_categories = [
    ((0.0, 0.4), "Dark"),
    ((0.4, 0.7), "Gray"),
    ((0.7, 1.0), "Bright"),
]

distance_categories = {
    "Dark": [
        ((0, 180), 1),
    ],
    "Gray": [
        ((0, 5), 0.1),
        ((5, 15), 0.85),
        ((15, 180), 1),
    ],
    "Bright": [
        ((0, 15), 0.25),
        ((15, 30), 0.75),
        ((30, 180), 1),
    ],
}

altitude_categories = [
    ((-90, 35), 0),
    ((35, 45), 0.95),
    ((45, 90), 1),
]

phase_scores = {
    "B|B": 0.5,
    "B|R": 0.8,
    "B|T": 0.4,
    "R|B": 0.8,
    "R|R": 0.9,
    "R|T": 1,
    "T|B": 0.4,
    "T|R": 1,
    "T|T": 0.1,
}

science_tl = tl.copy()
science_tl.target_list = science_tl.target_list[science_tl.target_list["Target Type"] == "QuadEB"]
pl = pr.PriorityList(science_tl, session, interval=60 * u.min)
pr.calculate_moon_priority(pl, illumination_categories=illumination_categories, dist_categories=distance_categories)
pr.calculate_altitude_priority(pl, altitude_categories=altitude_categories)
pr.calculate_list_priority(pl, "HQND", false_value=0.75)
pr.calculate_phase_priority(pl, phase_defs=phase_defs, phase_categories=phase_scores)
pr.calculate_overall_priority(pl)
pr.aggregate_target_priorities(pl, skip_column_threshold=0.3)
pl.categorize_priorities(
    bins=[0.00, 0.20, 0.40, 0.6, 1.00], labels=["", "*", "* *", "* * *"]
)

 '2025-09-29T04:00:00.000000000' '2025-09-29T05:00:00.000000000'
 '2025-09-29T06:00:00.000000000' '2025-09-29T07:00:00.000000000'
 '2025-09-29T08:00:00.000000000' '2025-09-29T09:00:00.000000000'
 '2025-09-29T10:00:00.000000000' '2025-09-29T11:00:00.000000000'
 '2025-09-29T12:00:00.000000000'], obsgeoloc=[(1659761.35991948, -5114828.14532952, 3423775.08698404),
 (2930577.90572714, -4511092.51088344, 3420565.46072497),
 (4001180.59569034, -3598248.50915968, 3417846.11727775),
 (4798211.88163603, -2438844.08844049, 3415803.3938731 ),
 (5267059.29109152, -1112321.48364356, 3414577.26747262),
 (5375597.46653513,   290426.15489587, 3414251.76293765),
 (5116389.39056476,  1673282.74782412, 3414849.19496306),
 (4507195.9693317 ,  2941495.14756309, 3416328.63836199),
 (3589759.0576503 ,  4008165.6112038 , 3418588.73156311),
 (2426941.31274085,  4800206.02213206, 3421474.62126905),
 (1098418.85343746,  5263345.87792678, 3424788.57246891)] m, obsgeovel=[( 372.98746346, 120.40527962, -0.93985921),

In [17]:
readme = lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session)
with open("UVa_Multistar.README", "w") as f:
    f.write(readme)

other_files = {
    "LBT Readme": lbt.write_lbt_readme_file("UVa_Multistar", tl.target_list, session),
    "LBT CSV": open("UVa_Multistar.csv", "r").readlines(),
}

html.render_observing_pages(tl, pl, other_files, html_dir)
# pl.categorical_priorities[0]

In [16]:
import asyncio
import glob
# for categorical_file in glob.glob("*.*"):

for categorical_file in glob.glob("../../Observing Files/LBT Observing 2025-07-04/Categorical Priorities *.html"):

    await html.html_to_pdf(categorical_file, categorical_file.replace(".html", ".pdf"))
    print(categorical_file)